# Trying to understand what metrics where used

In [ ]:
! pip install sympy

  Using cached https://files.pythonhosted.org/packages/dd/f6/ed485ff22efdd7b371d0dbbf6d77ad61c3b3b7e0815a83c89cbb38ce35de/sympy-1.3.tar.gz
    100% |████████████████████████████████| 522kB 5.3MB/s ta 0:00:011
  Running setup.py bdist_wheel for sympy ... done
  Stored in directory: /home/group7/.cache/pip/wheels/6c/59/86/478e3c0f298368c119095cc5985dedac57c0e35a85c737f823
  Running setup.py bdist_wheel for mpmath ... done
  Stored in directory: /home/group7/.cache/pip/wheels/63/9d/8e/37c3f6506ed3f152733a699e92d8e0c9f5e5f01dea262f80ad
Successfully built sympy mpmath


In [64]:
from pathlib import Path
import pandas as pd
import sklearn.metrics
from sympy import symbols, nonlinsolve

In [65]:
data ="""Persuasive   0.81 0.84 0.91
Audience   0.80 0.99 0.88
Agreement  0.69 0.85 0.76
Informative   0.76 0.74 0.75
Mean   0.74 0.78 0.75
Controversial   0.67 0.64 0.65
Disagreement  0.60 0.68 0.64
topic 0.62 0.67 0.61
Sentiment  0.44 0.46 0.43"""

In [66]:
arr = data.split('\n')

In [67]:
arr

['Persuasive   0.81 0.84 0.91',
 'Audience   0.80 0.99 0.88',
 'Agreement  0.69 0.85 0.76',
 'Informative   0.76 0.74 0.75',
 'Mean   0.74 0.78 0.75',
 'Controversial   0.67 0.64 0.65',
 'Disagreement  0.60 0.68 0.64',
 'topic 0.62 0.67 0.61',
 'Sentiment  0.44 0.46 0.43']

In [68]:
pr_val, re_val, f1_val = {}, {}, {}

In [69]:
for l in arr:
    name, prec, reca, f1sc = l.split()
    pr_val[name] = float(prec)
    re_val[name] = float(reca)
    f1_val[name] = float(f1sc)

In [90]:
def f1(p, r):
    return 2 * p * r * 1 / (p + r)

def prec(true_pos, false_pos):
    return true_pos / (false_pos + true_pos)

def reca(true_pos, false_neg):
    return true_pos  / (true_pos + false_neg)

In [91]:
# Persuasive
f1(0.81, 0.84) # 0.91 🛑

0.8247272727272728

In [92]:
# Audience
f1(0.80, 0.99) # 0.88 ✔️

0.8849162011173185

In [93]:
# Agreement w/ com
f1(0.69, 0.85) # 0.76 ✔️

0.7616883116883115

In [94]:
# Informative
f1(0.76, 0.74) # 0.75 ✔️

0.7498666666666667

In [95]:
# Mean
f1(0.74, 0.78) # 0.75 ✔️

0.7594736842105264

In [96]:
# controversial
f1(0.67, 0.64) # 0.65 ✔️

0.6546564885496183

In [97]:
# disagree
f1(0.60, 0.68) # 0.64 ✔️

0.6375000000000001

In [98]:
# off-topic 
f1(0.62, 0.67) # 0.61 🛑

0.644031007751938

In [99]:
# sentiment
f1(0.44, 0.46)  # 0.43 🛑

0.4497777777777778

In [100]:
pth = Path('/mnt/data/group07/johannes/ynacc_proc/replicate/split/val.csv')
df = pd.read_csv(pth)

In [133]:
def solve(name):
    counts = df['cl' + name.lower()].value_counts()
#     the reverse to ensure that there is only posibile solution

#         for i in reversed([[0, 1], [1, 0]]):
    all_res = []
    for i in [[0, 1], [1, 0]]:
        count_0 = counts[i[0]]
        count_1 = counts[i[1]]

        tp, tn, fp, fn = symbols('tp, tn, fp, fn', real=True)
        eq1 = count_1 - tp - fn
        eq2 = count_0 - tn - fp
        eq3 = prec(tp, fp) - pr_val[name]
        eq4 = reca(tp, fn) - re_val[name]

        system = [eq1, eq2, eq3, eq4]
        res = nonlinsolve(system, [tp, tn, fp, fn])
        print(res)
#         if len(res) > 0 and all(x > 0 for x in list(res)[0]):
        all_res.append(list(list(res)[0]) + [i])
    return all_res

In [134]:
def kappa(name):
    solutions = solve(name)
    print(solutions)
    for s in solutions:
        tp, tn, fp, fn, i = s
        y_true, y_pred = [], []

        for x in range(round(tp)):
            y_true.append(1)
            y_pred.append(1)

        for x in range(round(tn)):
            y_true.append(0)
            y_pred.append(0)

        for x in range(round(fp)):
            y_true.append(0)
            y_pred.append(1)

        for x in range(round(fn)):
            y_true.append(1)
            y_pred.append(0)
        print(i)
        print('kappa:  ', sklearn.metrics.cohen_kappa_score(y_true, y_pred))
        print('f1_mic: ', sklearn.metrics.f1_score(y_true, y_pred, average='micro'))
        print('f1_mac: ',sklearn.metrics.f1_score(y_true, y_pred, average='macro'))

In [135]:
for k in f1_val.keys():
    print(k)
    kappa(k)
    print()

Persuasive
{(71.4, 481.251851851852, 16.7481481481481, 13.6)}
{(418.32, -13.1244444444444, 98.1244444444444, 79.68)}
[[71.4000000000000, 481.251851851852, 16.7481481481481, 13.6000000000000, [0, 1]], [418.320000000000, -13.1244444444444, 98.1244444444444, 79.6800000000000, [1, 0]]]
[0, 1]
kappa:   0.7896017415802279
f1_mic:  0.9468267581475128
f1_mac:  0.8947953594234788
[1, 0]
kappa:   -0.17343597911689246
f1_mic:  0.7013422818791947
f1_mac:  0.41222879684418146

Audience
{(378.18, 104.455, 94.545, 3.82)}
{(197.01, 332.7475, 49.2525, 1.99)}
[[378.180000000000, 104.455000000000, 94.5450000000000, 3.82000000000000, [0, 1]], [197.010000000000, 332.747500000000, 49.2525000000000, 1.99000000000000, [1, 0]]]
[0, 1]
kappa:   0.5751386806319849
f1_mic:  0.8296041308089501
f1_mac:  0.7808674781416081
[1, 0]
kappa:   0.8155406288713061
f1_mic:  0.9122203098106713
f1_mac:  0.9071317756569979

Agreement
{(57.8, 489.031884057971, 25.968115942029, 10.2)}
{(437.75, -128.670289855072, 196.67028985507

In [107]:
kappa('topic')

{(268.0, 18.741935483871, 164.258064516129, 132.0)}
kappa:   -0.23743689765947673
f1_mic:  0.4922813036020583
f1_mac:  0.3790016121602948


In [108]:
from sympy.solvers.solveset import nonlinsolve
from sympy.core.symbol import symbols

In [19]:
# binary
tp, tn, fp, fn = symbols('tp, tn, fp, fn', real=True)
eq1 = 188 - tp - fn
eq2 = 395 - tn - fp
eq3 = prec(tp, fp) - 0.67
eq4 = reca(tp, fn) - 0.64
eq5 = f1(eq3, eq4) - 0.65

system = [eq1, eq2, eq3, eq4]
nonlinsolve(system, [tp, tn, fp, fn])

{(120.32, 215.417910447761, 179.582089552239, 67.68)}

In [138]:
# micro
a, b, c, d = symbols('a, b, c, d', real=True)
eq1 = 188 - a - b
eq2 = 395 - c - d
eq3 = prec(a + d, b + c) - 0.67
eq4 = reca(a + d, b + c) - 0.64
# eq5 = f1(eq3, eq4) - 0.65

system = [eq1, eq2, eq3, eq4]
nonlinsolve(system, [a, b, c, d])

EmptySet()

In [139]:
# macro
a, b, c, d = symbols('a, b, c, d', real=True)
eq1 = 188 - a - b
eq2 = 395 - c - d
eq3 = (prec(a, c) + prec(d, b)) / 2 - 0.67
eq4 = (reca(a, b) + reca(d, c)) / 2 - 0.64
eq5 = (f1(prec(a, c), reca(a, b)) + f1(prec(d, b), reca(d, c))) / 2 - 0.65

system = [eq1, eq2, eq3, eq4]
nonlinsolve(system, [a, b, c, d])

{(-0.67*d - 170.65*sqrt(-1.89242129226345e-5*d**2 + 0.005711568519704*d + 1.0) + 358.65, 0.67*d + 170.65*sqrt(-1.89242129226345e-5*d**2 + 0.005711568519704*d + 1.0) - 170.65, -d + 395, d), (-0.67*d + 170.65*sqrt(-1.89242129226345e-5*d**2 + 0.005711568519704*d + 1.0) + 358.65, 0.67*d - 170.65*sqrt(-1.89242129226345e-5*d**2 + 0.005711568519704*d + 1.0) - 170.65, -d + 395, d)}

In [141]:
# weighted
a, b, c, d = symbols('a, b, c, d', real=True)
eq1 = 188 - a - b
eq2 = 395 - c - d
eq3 = ((a + b) * prec(a, c) + (c + d) * prec(d, b)) / (a + b + c + d) - 0.67
eq4 = ((a + b) * reca(a, b) + (c + d) * reca(d, c)) / (a + b + c + d) - 0.64
eq5 = ((a + b) * f1(prec(a, c), reca(a, b)) + (c + d) * f1(prec(d, b), reca(d, c))) / (a + b + c + d) - 0.65

system = [eq1, eq2, eq3, eq4]
nonlinsolve(system, [a, b, c, d])

{(-14.4493657422819, 202.449365742282, 7.43063425771811, 387.569365742282), (582.023911196827, -394.023911196827, 603.903911196827, -208.903911196827)}

![](https://i.stack.imgur.com/M2LD0.png)